In [1]:
# Importing library
import pandas as pd
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import math 
import mysql.connector as msql
import re
import requests

In [2]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
# Taking files from define path which are in csv format
path = r''

# Change the current working directory to the specified path
os.chdir(path)

# Use glob to get a list of file paths for all CSV files in the current directory
file_paths = glob.glob('*.csv')

In [4]:
# Create an empty list to store the keywords
keywords = []

# Create an empty dictionary to store the dataframes
dataframes = {}

# Iterate through each file path in the list of file paths
for file in file_paths:
    # Find the starting position of the keyword by searching for 'Assignment_'
    start = file.find('Assignment_') + len('Assignment_')
    
    # Find the ending position of the keyword by searching for '.csv'
    end = file.rfind('.csv')
    
    # Extract the keyword from the file path, removing leading/trailing spaces, and replacing spaces with underscores
    keyword = file[start:end].strip().replace(' ', '_')
    
    # Add the extracted keyword to the list of keywords
    keywords.append(keyword)
    
    # Read the CSV file and store it in a dataframe with the keyword as the key in the dataframes dictionary
    dataframes[keyword] = pd.read_csv(file)


In [5]:
# List of keyword of our csv file present in our folder
keywords

['Career_Action_Plan_(CAP)',
 'Career_Exploration',
 'CV__Resume',
 'Goal_Setting',
 'LinkedIn',
 'Searching_Internship',
 'SMART_goals']

In [6]:
# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Assign the corresponding dataframe to a variable with the same name as the keyword
    # This is done by using the globals() function to create a new variable in the global namespace
    globals()[keyword] = dataframes[keyword]

    # We have use golabl function to create dataframe of keyword so it should take name of dataframe as keyword. Like Swot will be name of datafarme.

In [ ]:
# Reading Kalpana SHE for STEM Incubator file from Source files

directory_path =(r"")

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    Kalpana = pd.read_csv(file_path,usecols=['Email'])
    print(f"Data from {file}:")

Data from She for STEM - Uttarakhand.csv:


In [9]:
# Create an empty dataframe to store the final results
final_df = pd.DataFrame()

# Create a dictionary to store comments for each keyword
comments_dict = {}

# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Filter the columns based on specific conditions...
    filtered_columns = [col for col in globals()[keyword] if ('message' in col) or ('email' in col) or (col == 'status')]

    new_df = globals()[keyword][filtered_columns]
    
    message_columns = [col for col in new_df.columns if 'message' in col]
    new_df[message_columns] = new_df[message_columns].fillna('')
    
    # Create a new 'Comment' column by joining the values of the 'message' columns for each row
    comment_col_name = 'Comment_' + keyword
    new_df[comment_col_name] = new_df[message_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    
    # Store the comments column name in the dictionary
    comments_dict[keyword] = comment_col_name
    
    # Modify the 'Email' column by removing a specific prefix from the values
    new_df['Email'] = new_df['user/email'].str.replace('vigyanshaalainternational1617-', '')
    
    # Rename the 'status' column to the keyword
    new_df = new_df.rename(columns={'status': keyword})
    
    # Select the desired columns for the new dataframe
    selected_columns = ['Email', keyword, comment_col_name]
    new_df = new_df[selected_columns]
    
    # If the final dataframe is empty, assign it as the new_df
    if final_df.empty:
        final_df = new_df.copy()
    # If the final dataframe is not empty, merge the new_df with the existing final dataframe using an outer join
    else:
        final_df = pd.merge(final_df, new_df, on='Email', how='outer', suffixes=('', f'_{keyword}'))

# Fill NaN values in the final dataframe
final_df = final_df.fillna('')

# Now, final_df contains all unique emails from all keywords


In [10]:
final_df.shape

(165, 15)

In [11]:
final_df.columns

Index(['Email', 'Career_Action_Plan_(CAP)', 'Comment_Career_Action_Plan_(CAP)',
       'Career_Exploration', 'Comment_Career_Exploration', 'CV__Resume',
       'Comment_CV__Resume', 'Goal_Setting', 'Comment_Goal_Setting',
       'LinkedIn', 'Comment_LinkedIn', 'Searching_Internship',
       'Comment_Searching_Internship', 'SMART_goals', 'Comment_SMART_goals'],
      dtype='object')

In [12]:
# Create a list of columns to assign, excluding 'Email' and columns starting with 'Comment'
assign = [col for col in final_df.columns if col != 'Email' and not col.startswith('Comment')]


In [13]:
# Remove leading/trailing spaces in the selected columns
final_df[assign] = final_df[assign].applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [14]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['Goal_Setting'].unique()

array(['reviewed', 'under review', '', 'rejected'], dtype=object)

In [15]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['Goal_Setting'].unique()

array(['reviewed', 'under review', '', 'rejected'], dtype=object)

### If you get new unique values in our column then add marks to it. such example "Pass" then at Please add pass in below dict and give marks to it like replace_dict = {'under review': 30, 'reviewed': 30,'rejected': 80, 'Pass': 100}

# Below you can give marks for Assignment 👇⬇️

In [16]:
replace_dict = {'under review': 30, 'reviewed': 100,'rejected': 80,'': 0}


# 👆☝️


In [17]:
# Replace values in selected columns
final_df[assign] = final_df[assign].replace(replace_dict)

In [18]:
# Convert the selected columns to float
final_df[assign] = final_df[assign].astype(float)

In [19]:
# Filling NaN Values with zero
final_df[assign]=final_df[assign].fillna(0)

In [20]:
# Calculate the sum of selected columns
sum_of_scores = final_df[assign].sum(axis=1)

# Count the number of selected columns
num_of_columns = len(assign)

# Calculate the average score
final_df['Assignment_Score'] = sum_of_scores / num_of_columns


In [21]:
# Rounding Off values of Assignment score to two digits
final_df['Assignment_Score']=final_df['Assignment_Score'].round(2)

In [22]:
# Filling NaN Values with empty
final_df=final_df.fillna('')

In [23]:
final_df = final_df.replace('Successfully Submitted ', '# ', regex=True)


In [25]:
# Iterate through all columns in the dataframe
for col in final_df.columns:
    # Check if column name contains '.'
    if '.' in col:
        # Replace '.' with '_' and remove ' ' only where '.' is present in the column name
        new_col = col.replace('.', '_').replace(' ', '') if '.' in col else col
        # Rename the column
        df.rename(columns={col: new_col}, inplace=True)


In [26]:
# Cheaking the columns present in the dataframe
final_df.columns

Index(['Email', 'Career_Action_Plan_(CAP)', 'Comment_Career_Action_Plan_(CAP)',
       'Career_Exploration', 'Comment_Career_Exploration', 'CV__Resume',
       'Comment_CV__Resume', 'Goal_Setting', 'Comment_Goal_Setting',
       'LinkedIn', 'Comment_LinkedIn', 'Searching_Internship',
       'Comment_Searching_Internship', 'SMART_goals', 'Comment_SMART_goals',
       'Assignment_Score'],
      dtype='object')

In [27]:
final_df = pd.merge(final_df, Kalpana, on='Email', how='inner')


In [28]:
final_df.shape

(82, 16)

### The step remaing here is renaming i.e while putting to our mysql

In [29]:
# Get the list of column names in the final_df
column_names = final_df.columns

# Iterate through the column names and rename columns starting with "Comment"
for col in column_names:
    if col.startswith("Comment"):
        new_col_name = "Comment"
        final_df = final_df.rename(columns={col: new_col_name})



In [30]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan_(CAP)', 'Comment', 'Career_Exploration',
       'Comment', 'CV__Resume', 'Comment', 'Goal_Setting', 'Comment',
       'LinkedIn', 'Comment', 'Searching_Internship', 'Comment', 'SMART_goals',
       'Comment', 'Assignment_Score'],
      dtype='object')

# Below insted of renaming name if at staring only we have taken the assignment name as we want mention in txt in source file we will not need to do below step 👇⬇️

In [31]:
# Renaming the column to put in datafarme 
new_names = {
    'SWOT': 'SWOT',
    'Career_Action_Plan_(CAP)': 'Career_Action_Plan',
    'Goal_Setting' : 'Goal_Setting',
    'Critical_thinking': 'Critical_Thinking',
    'Masteclass_1_-_Career_Exploration': 'Career_Exploration',
    'CV__Resume': 'CV_Resume',
    'LinkedIn': 'LinkedIn_Profile',
    'Planning_&_Applying_for_Masters_in_India_&_Abroad': 'Planning_Masters',
    'RIASEC_personality_test': 'RIASEC',
    'SMART_goal': 'SMART_Goal',
    'Assignment_Caree' : 'Career_Exploration',
    'Career_Exploration' : 'Career_Exploration',
    'SMART_goals' : 'SMART_Goal',
    'Searching_Internship':'Internship_Searching'
}

for col in final_df.columns:
    if col in new_names:
        final_df.rename(columns={col: new_names[col]}, inplace=True)


In [32]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comment', 'Career_Exploration',
       'Comment', 'CV_Resume', 'Comment', 'Goal_Setting', 'Comment',
       'LinkedIn_Profile', 'Comment', 'Internship_Searching', 'Comment',
       'SMART_Goal', 'Comment', 'Assignment_Score'],
      dtype='object')

In [33]:
# Renaming the 'comment' in our given format
cols = final_df.columns.tolist()
new_cols = []
prev_col = ''

for col in cols:
    if col == 'Comment':
        new_cols.append(f'Comments_{prev_col}')
    else:
        new_cols.append(col)
    prev_col = col

final_df.columns = new_cols


In [34]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comments_Career_Action_Plan',
       'Career_Exploration', 'Comments_Career_Exploration', 'CV_Resume',
       'Comments_CV_Resume', 'Goal_Setting', 'Comments_Goal_Setting',
       'LinkedIn_Profile', 'Comments_LinkedIn_Profile', 'Internship_Searching',
       'Comments_Internship_Searching', 'SMART_Goal', 'Comments_SMART_Goal',
       'Assignment_Score'],
      dtype='object')

In [35]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
final_df['Email'] = final_df['Email'].apply(clean_email)

# Adding Genral Info 

In [ ]:
# Import Genral Info
gi = pd.read_csv(r"")

In [37]:
gi = gi[['Email', 'Name', 'Phone', 'Name_of_College_University','Currently_Pursuing_Degree']].copy()

In [38]:
# Merge with gi
output = pd.merge(gi, final_df, on='Email', how='outer')

In [39]:
output.columns

Index(['Email', 'Name', 'Phone', 'Name_of_College_University',
       'Currently_Pursuing_Degree', 'Career_Action_Plan',
       'Comments_Career_Action_Plan', 'Career_Exploration',
       'Comments_Career_Exploration', 'CV_Resume', 'Comments_CV_Resume',
       'Goal_Setting', 'Comments_Goal_Setting', 'LinkedIn_Profile',
       'Comments_LinkedIn_Profile', 'Internship_Searching',
       'Comments_Internship_Searching', 'SMART_Goal', 'Comments_SMART_Goal',
       'Assignment_Score'],
      dtype='object')

In [40]:
# Rearrange the columns
new_column_order = [
    'Email', 'Name', 'Phone', 'Name_of_College_University',
    'Currently_Pursuing_Degree', 'Goal_Setting', 'Comments_Goal_Setting', 
    'SMART_Goal', 'Comments_SMART_Goal', 'CV_Resume', 
    'Comments_CV_Resume', 'Career_Exploration', 
    'Comments_Career_Exploration', 'Career_Action_Plan', 
    'Comments_Career_Action_Plan','Internship_Searching',
    'Comments_Internship_Searching','LinkedIn_Profile',
    'Comments_LinkedIn_Profile', 'Assignment_Score'
]

output = output[new_column_order]

In [41]:
#final_df.to_csv('Career_Exploration_Withcomment.csv', mode='a',index=False)

In [ ]:
output.to_csv(r"")

In [43]:
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comments_Career_Action_Plan',
       'Career_Exploration', 'Comments_Career_Exploration', 'CV_Resume',
       'Comments_CV_Resume', 'Goal_Setting', 'Comments_Goal_Setting',
       'LinkedIn_Profile', 'Comments_LinkedIn_Profile', 'Internship_Searching',
       'Comments_Internship_Searching', 'SMART_Goal', 'Comments_SMART_Goal',
       'Assignment_Score'],
      dtype='object')

In [44]:
# Renaming the column to put in datafarme 
new_names = {
    'Assignment_CAP': 'Career_Action_Plan',
    'Comments_Assignment_CAP': 'Comments_Career_Action_Plan',
    'Assignment_Career_Exploration' : 'Career_Exploration',
    'Comments_Assignment_Career_Exploration': 'Comments_Career_Exploration',
    'Assignment_Goal_Setting': 'Goal_Setting',
    'Comments_Goal_Setting': 'Comments_Goal_Setting',    
    'Goal_Setting': 'Goal_Setting',
    'Comments_Goal_Setting': 'Comments_Goal_Setting',
    'Assignment_LinkedIn': 'LinkedIn_Profile',
    'Comments_Assignment_LinkedIn' : 'Comments_LinkedIn_Profile',
    'Planning_&_Applying_for_Masters_in_India_&_Abroad': 'Planning_Masters',
    'RIASEC_personality_test': 'RIASEC',
    'SMART_goals': 'SMART_Goal',
    'Assignment_Searching_&_Securing_Internship' : 'Internship_Searching',
    'Comments_Assignment_Searching_&_Securing_Internship' : 'Comments_Internship_Searching',
    'Assignment_Resume' : 'CV_Resume',
    'Comments_Assignment_Resume' : 'Comments_CV_Resume',
    'Assignment_SMART_goals' : 'SMART_Goal',
    'Comments_SMART_goals' : 'Comments_SMART_Goal',
    'CV__Resume' :'CV_Resume',
    'Comments_CV__Resume' : 'Comments_CV_Resume'
}

for col in final_df.columns:
    if col in new_names:
        final_df.rename(columns={col: new_names[col]}, inplace=True)

In [45]:
final_df.columns

Index(['Email', 'Career_Action_Plan', 'Comments_Career_Action_Plan',
       'Career_Exploration', 'Comments_Career_Exploration', 'CV_Resume',
       'Comments_CV_Resume', 'Goal_Setting', 'Comments_Goal_Setting',
       'LinkedIn_Profile', 'Comments_LinkedIn_Profile', 'Internship_Searching',
       'Comments_Internship_Searching', 'SMART_Goal', 'Comments_SMART_Goal',
       'Assignment_Score'],
      dtype='object')

# Storing data on MySQL

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')
cursor =conn.cursor()

☝️ REMOVE COMMENT OF ABOVE I.E REMOVE #

In [47]:
# Your existing code for inserting data into the database table
for i, row in final_df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(final_df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO she_for_stem.11_incubator_assignment_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in final_df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))


In [48]:
conn.commit()

# Storing Data in Supabase

In [ ]:
# Supabase URL and API key
url = ''
api_key = ''


# Headers for the request
headers = {
    'apikey': api_key,
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
    'Prefer': 'resolution=merge-duplicates'  # Enable upsert functionality
}


In [50]:
table_name = '11_incubator_assignment_monitoring'

# Batch size for upserting
batch_size = 1000  # You can adjust this value based on your performance needs

# List to store rows before sending them in batches
batch_data = []

# Iterate through each row in the DataFrame
for i, row in final_df.iterrows():
    # Replace NaN values with None
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row]
    # Convert row to a dictionary
    row_dict = dict(zip(final_df.columns, row))
    
    # Add the row to the batch
    batch_data.append(row_dict)
    
    # If the batch size is reached, send the data
    if len(batch_data) >= batch_size:
        # Send a batch of rows
        response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
        
        # Check response
        if response.status_code in [200, 201]:
            print(f'Batch of {len(batch_data)} rows upserted successfully')
        else:
            print(f'Failed to upsert batch: {response.status_code}, {response.text}')
        
        # Clear the batch after sending
        batch_data = []

# Send any remaining rows in the last batch
if batch_data:
    response = requests.post(f'{url}/rest/v1/{table_name}', headers=headers, json=batch_data)
    
    if response.status_code in [200, 201]:
        print(f'Final batch of {len(batch_data)} rows upserted successfully')
    else:
        print(f'Failed to upsert final batch: {response.status_code}, {response.text}')


Final batch of 82 rows upserted successfully
